## <b><font color='darkblue'>L1: Build your first agent</font></b>
([source](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/9in9mv/build-your-first-agent)) <font size='3ptx'><b>Welcome to Building AI News Podcast Agent with Google ADK! In this lesson, you'll build your first AI agent and give it the power to access real-time information from the web.</b> You'll learn the fundamental structure of an agent and explore the ADK Web UI, which is a convenient way to trace your agent's thinking and interact through live voice conversations.</font>

Throughout this course, you'll be building a complete AI News Podcast Agent that can research the latest AI developments and generate professional audio podcasts. This lesson lays the foundation with a simple agent that can fetch recent AI news from the web.

### <b><font color='darkgreen'>What you'll learn</font></b>
In this lesson, you'll build a simple agent that can fetch recent AI news from the web. An agent is a construct that has an LLM and tools:
- You'll learn about the fundamental structure of an agent
- You'll become familiar with an LLM as the brain of the agent providing it the generative language capabilities
- You'll explore tools that let the agent take actions in the real world
- You'll also explore alternative development approaches including YAML configuration and Web Builder options.

### <b><font color='darkgreen'>1.1 Getting started</font></b>
The following python packages are pre-installed, but listed here for your reference. This will install a handy set of command-line tools that will be useful when working with ADK.

In [3]:
# !pip install -q google-adk>=1.12.0

In [4]:
!pip freeze | grep google-adk

google-adk==1.17.0


In [14]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

### <b><font color='darkgreen'>1.2 Setting up your first agent</font></b>
<font size='3ptx'><b>Before we dive into building agents, let's set up a new folder structure with ADK's built-in project scaffolding using the [`adk create`](https://google.github.io/adk-docs/api-reference/cli/cli.html#adk-create) command.</b></font>

When you run `adk create`, it generates three essential files:
1. The `.env` file securely stores your API credentials and configuration.
2. The `__init__.py` file marks the directory as a Python package, nabling proper imports.
3. Most importantly, the `agent.py` file provides a clean foundation where you'll implement your agent.

File structure:
```
app_01/
    __init__.py
    agent.py
    .env
```

[**ADK create**](https://google.github.io/adk-docs/api-reference/cli/cli.html#adk-create) supports two project types. The `--type=code` option generates a Python-based agent in `agent.py`. The `--type=config` option creates a YAML-based agent configuration.

The `--model` parameter specifies the LLM to be used by the agent. We will override this and experiment with different models as we create the agents in this lesson.

#### <b>Setting up authentication</b>
<b>Before using any AI models, the first step is to configure API credentials</b>. There are several methods to authenticate with Google's Gemini models, including:
1. Google API key and
2. Vertex AI based authentication.

For more information on setting up your own authentication, visit Google.

The `--api_key` parameter automatically configures your project for Google's Gemini API, setting up the proper authentication environment variables. You can go to [**Google AIStudio**](https://aistudio.google.com/api-keys) to generate your own key. After that, create an environment variables as below or keep them in your `.env` for development:
```shell
GOOGLE_GENAI_USE_VERTEXAI=0
GOOGLE_API_KEY=<YOUR_GENERATED_KEY>
```

Run the cell below to create the folder structure for your agent!

In [15]:
# Load setting from `.env`
load_dotenv()

!rm -rf app_01/
!adk create --type=code app_01 --model gemini-2.0-flash-live-001 --api_key $GOOGLE_API_KEY


Agent created in /home/john/Gitrepos/ml_articles/deeplearning_ai/building_live_voice_agents_with_googles_adk/app_01:
- .env
- __init__.py
- agent.py



### <b><font color='darkgreen'>1.3 Writing your first `agent.py`</font></b>
<font size='3ptx'><b>Throughout this course, you will be using the adk create command to create folders and then write to its `agent.py` using the cell magic in the notebook.</b> Cell magic uses specific commands to interact with the files in your new agent folder.</font>

You will use `%%writefile FILENAME` to do this.

Let's start with the absolute simplest agent possible—one that has no internet access at all. You'll create an Agent with a unique name, specify the LLM model, and give it basic instructions.

<b>As with any agent, you need an LLM to start with. ADK is model agnostic - meaning you can provide it with any model of your choice like Gemini, Claude, Ollama and even use LiteLLM to bring in other models</b>. In this lesson, you'll be using the Gemini 2.0 flash and Gemini 2.0 flash live models.

In [16]:
%%writefile app_01/agent.py

from google.adk.agents import Agent

root_agent = Agent(
    name="ai_news_agent_simple",
    model="gemini-2.0-flash-live-001", # Essential for live voice interaction
    instruction="You are an AI News Assistant.",
)

Overwriting app_01/agent.py


### <b><font color='darkgreen'>1.4 Running your first agent</font></b>
<font size='3ptx'><b>Now comes the exciting part—actually using what you've built</b>. ADK gives you several ways to run your agent, but for development and learning, you'll use the ADK Web UI.</font>

To start the local server and access the Web UI, follow the below steps:
Terminal Instructions:
1. To open the terminal, run the cell below.
2. Change to the current lesson directory with the following command: `cd L1`
3. Start your agent using the following command: `adk web --host 0.0.0.0 --port 8001`
4. Run the cell below the terminal, to display the proxy URL. (<font color='brown'>This step is only needed within this codelab. When running externally, you can directly access the local server link.</font>)

Here's a closer look at the command that starts the agent.
- `adk web`: A simple command that starts your agent on the Web UI
- `--host 0.0.0.0`: The host address
- `--port 8001`: The port we will access the server from. In this case, port 8001.

In [25]:
# Run `adk web --host 0.0.0.0 --port 8001` first.
# start a new terminal
import os
from IPython.display import IFrame

os.environ['DLAI_LOCAL_URL'] = 'http://192.168.1.125:8001/dev-ui/?app=app_01'
IFrame(f"{os.environ.get('DLAI_LOCAL_URL')}", 
       width=1200, height=600)

### <b><font color='darkgreen'>1.5 Adding tools to your agent</font></b>
<font size='3ptx'><b>But here's the problem</b>: try asking this agent about the latest AI developments, and you'll quickly discover it can only tell you about things that happened before its training cutoff date. For an AI news assistant that's supposed to fetch the latest news, that's not particularly helpful.</font>

You need to fix that by providing your agent with [**Tools**](https://google.github.io/adk-docs/tools/). Create another empty scaffolding folder to create an agent with a tool by running the cell below.

In [26]:
!adk create --type=code app_02 --model gemini-2.0-flash --api_key $GOOGLE_API_KEY


Agent created in /home/john/Gitrepos/ml_articles/deeplearning_ai/building_live_voice_agents_with_googles_adk/app_02:
- .env
- __init__.py
- agent.py



In [27]:
# %load app_02/agent.py
from google.adk.agents.llm_agent import Agent

root_agent = Agent(
    model='gemini-2.0-flash',
    name='root_agent',
    description='A helpful assistant for user questions.',
    instruction='Answer user questions to the best of your knowledge',
)


#### <b>Adding Google Search Tool</b>
In this scenario, to fetch the latest news, let's provide your agent with a built-in tool, [**google_search**](https://google.github.io/adk-docs/tools/built-in-tools/#google-search). These built-in tools come pre-packed with the library.

To add it to the agent, just import it and provide it as a tool in the tools array. Run the cell below to import it.

In [28]:
%%writefile app_02/agent.py

from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="ai_news_agent_simple",
    model="gemini-2.0-flash", # Essential for live voice interaction
    instruction="You are an AI News Assistant. Use Google Search to find recent AI news.",
    tools=[google_search]
)

Overwriting app_02/agent.py


**Refresh the ADK Web UI**. You should see a new app name (app_02) in the dropdown. Select it and start talking to your agent.

Let's test the agent with the Google search tool by asking a query "What is the latest AI News?". Watch what happens: Your agent will use the Google Search tool to find current information, process the results, and give you a comprehensive, up-to-date response with sources.

Just like that, your agent can now access real-time information from across the web!

ADK comes with several other powerful built-in tools—there are tools for running your code in a sandbox, querying databases, even integrations with Google Workspace tools like Calendar, Drive etc.

### <b><font color='darkgreen'>1.7 Alternative development approaches</font></b>
<font size='3ptx'><b>ADK supports a declarative approach using YAML files</b>. This is ideal for when you want rapid prototyping without code or want configuration or declarative style development.</font>

Create a new folder for the agent by running the cell below. Notice that this time you use `--type=config`. This will create a new directory for your agent.
```
app_04/
    __init__.py
    root_agent.yaml
    .env
```

A new file for your agent called <font color='olive'>root_agent.yaml</font> is created and you will use that to configure your agent.

In [29]:
# You can also define the agent with simple YAML confirguration 

!adk create --type=config app_04 --model gemini-2.5-flash --api_key $GOOGLE_API_KEY


Agent created in /home/john/Gitrepos/ml_articles/deeplearning_ai/building_live_voice_agents_with_googles_adk/app_04:
- .env
- __init__.py
- root_agent.yaml



In [30]:
# %load app_04/root_agent.yaml
# yaml-language-server: $schema=https://raw.githubusercontent.com/google/adk-python/refs/heads/main/src/google/adk/agents/config_schemas/AgentConfig.json
name: root_agent
description: A helpful assistant for user questions.
instruction: Answer user questions to the best of your knowledge
model: gemini-2.5-flash


#### <b>Building agents with the YAML configuration</b>
The config below maps directly to our Python version. You define the same core components—name, model, instructions, and tools—but in a declarative format. The schema validation ensures your configuration is correct.

In [31]:
%%writefile app_04/root_agent.yaml

name: ai_news_agent_yaml
model: gemini-2.0-flash-live-001 # Using a model that supports live voice
description: An agent that answers questions about recent news in Artificial Intelligence.
instruction: You are an AI News Assistant. Use Google Search to find and summarize recent AI news.
tools:
  - name: google_search

Overwriting app_04/root_agent.yaml


Now, **refresh the ADK Web UI** and switch to the yaml version in the UI (<font color='brown'>select `app_04` in the dropdown</font>). Try interacting with the agent using voice and verify it is fully functional.

### <b><font color='darkgreen'>1.8 Fine-tuning agent instructions</font></b>
<font size='3ptx'><b>So far your agent has simple instructions, but for reliable behavior, you need more sophisticated instruction engineering</b>. Let's enhance your agent with strict behavioral controls.</font>

Create a new folder using the [**adk create**](https://google.github.io/adk-docs/api-reference/cli/cli.html#adk-create) command.

In [32]:
# You can also modify the instructions  

!adk create --type=code app_05 --model gemini-2.0-flash --api_key $GOOGLE_API_KEY


Agent created in /home/john/Gitrepos/ml_articles/deeplearning_ai/building_live_voice_agents_with_googles_adk/app_05:
- .env
- __init__.py
- agent.py



In [33]:
# %load app_05/agent.py
from google.adk.agents.llm_agent import Agent

root_agent = Agent(
    model='gemini-2.0-flash',
    name='root_agent',
    description='A helpful assistant for user questions.',
    instruction='Answer user questions to the best of your knowledge',
)


Write the updated agent code to the `agent.py` file:

In [34]:
%%writefile app_05/agent.py

from google.adk.agents import Agent
from google.adk.tools import google_search

root_agent = Agent(
    name="ai_news_agent_strict",
    model="gemini-2.0-flash-live-001",
    instruction="""
    **Your Core Identity and Sole Purpose:**
    You are a specialized AI News Assistant. Your sole and exclusive purpose is to find and summarize recent news (from the last few weeks) about Artificial Intelligence.

    **Strict Refusal Mandate:**
    If a user asks about ANY topic that is not recent AI news, you MUST refuse.
    For off-topic requests, respond with the exact phrase: "Sorry, I can't answer anything about this. I am only supposed to answer about the latest AI news."

    **Required Workflow for Valid Requests:**
    1. You MUST use the `google_search` tool to find information.
    2. You MUST base your answer strictly on the search results.
    3. You MUST cite your sources.
    """,
    tools=[google_search]
)

Overwriting app_05/agent.py


#### <b>Understanding Advanced Instructions</b>
This enhanced instruction pattern includes:
- Clear Identity: Explicitly defines the agent's sole purpose
- Refusal Mechanism: Provides exact phrases for rejecting off-topic requests
- Workflow Requirements: Forces the agent to use tools and cite sources
- Behavioral Boundaries: Sets expectations for valid vs. invalid requests

This creates a much more reliable and focused agent behavior.

#### <b>Testing Instructions</b>
**Refresh the ADK Web UI**, select the appropriate app (`app_05`) from the dropdown menu and test your enhanced agent with both valid and invalid requests:

* **Valid prompts** (should get a response):
  - "What's the latest AI news about Google?"
  - "Tell me about recent AI chip developments"
* **Invalid prompts** (should be refused):
  - "What's the weather today?"
  - "Help me with my homework"

Watch how the agent now maintains strict boundaries while still being helpful for AI-related queries.

### <b><font color='darkgreen'>CleanUp</font></b>
After finishing, make sure to run the cell below to close your connection so it does not interfere as you progress through the notebooks.

In [35]:
# Kill ADK process
!pkill -f "adk web"

## <b><font color='darkblue'>L3: Tools for your agent</font></b>
([source](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/1541po/tools-for-your-agent)) <font size='3ptx'>Welcome to Lesson 3 Tools for your agent. <b>In this lesson, you'll learn how to add custom function tools and enhance your agent with external data sources</b>. You'll integrate financial data APIs and build more sophisticated agent instructions—essential skills for building reliable AI applications.</font>

## <b><font color='darkblue'>L4: Adding a Research Agent</font></b>
([source](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/y8oy6u/adding-a-research-agent)) <font size='3ptx'><b>In this lesson, you'll transform your agent from a conversational interface into a research coordinator that works behind the scenes.</b> You'll learn to implement a [<b>coordinator-dispatcher pattern</b>](https://google.github.io/adk-docs/agents/multi-agents/#coordinatordispatcher-pattern) where the root agent delegates research work to background processes and saves structured reports for later use.</font>

## <b><font color='darkblue'>L5: Instruction Tuning and Guardrails</font></b>
([source](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/moimyh/instruction-tuning-and-guardrails)) <font size='3ptx'><b>In this lesson, you'll take everything you've learned from previous lessons and add advanced control mechanisms that transform your agents from helpful assistants into specialized, production-ready systems</b>.</font>

## <b><font color='darkblue'>L6: Multi-agent orchestration</font></b>
([source](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/dewdnp/multi-agent-orchestration)) <font size='3ptx'><b>In this lesson you'll learn production-grade multi-agent patterns where you'll refactor your system into a three-tier architecture with proper separation of concerns.</b></font>

## <b><font color='darkblue'>Supplement</font></b>
* [**L2: ADK primitives - Session, State and Memory**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/hl2hjq/adk-primitives---session%2C-state%2C-and-memory)
* [**L3: Tools for your agent**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/1541po/tools-for-your-agent)
* [**L4: Adding a Research Agent**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/y8oy6u/adding-a-research-agent)
* [**L5: Instruction Tuning and Guardrails**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/moimyh/instruction-tuning-and-guardrails)
* [**L6 Multi-agent orchestration**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/dewdnp/multi-agent-orchestration)
* [**L7: Productionize your agent**](https://learn.deeplearning.ai/courses/building-live-voice-agents-with-googles-adk/lesson/txqtf4/%5Boptional%5D-productionize-your-agent)